# A homemade stream cipher

In this notebook we will use a pseudorandom generator to build our first homemade cipher

# Table of contents:

* [Encoding a message to bytes](#encoding)
* [Stream cipher](#stream)
    * [XORing bytes](#xor)
    * [Bytes Random Generator](#brg)
* [Practical demonstration with python objects](#practical)
    
Author: [Sebastià Agramunt Puig](https://github.com/sebastiaagramunt) for [OpenMined](https://www.openmined.org/) Privacy ML Series course.



## Encoding a message to bytes <a class="anchor" id="elgamal"></a>

To code our first cipher based on a pseudorandom generator we need to understand a little bit better how information is encoded in python. For this, we coded several functions to help us.

In [1]:
from crypto import bytes_to_bin, bytes_to_hex

message = b"simple message"

bin_repr = bytes_to_bin(message, pre="")
hex_repr = bytes_to_hex(message, pre="")

print(f"message:\n{message}\nlen bytes: {len(message)}\n")
print(f"message in binary:\n{bin_repr}\nlen bits: {len(bin_repr)}\n")
print(f"message in hexadecimal:\n{hex_repr}\nlen hex:{len(hex_repr)}\n")

message:
b'simple message'
len bytes: 14

message in binary:
0111001101101001011011010111000001101100011001010010000001101101011001010111001101110011011000010110011101100101
len bits: 112

message in hexadecimal:
73696d706c65206d657373616765
len hex:28



## Stream Cipher <a class="anchor" id="stream"></a>

In our first cipher we are going to pad our messeage (XOR) with a pseudorandom "stream" of bytes. This kind of cipher is known as stream cipher.

<img src="img/stream_cipher.png" style="width:900px"/>

### XORing bytes <a class="anchor" id="xor"></a>

We need to implement the XOR operation byte-wise, that is how we "pad" our message to convert it to its encrypted form.

In [2]:
from crypto import binary, hexadecimal
import random

a = random.randrange(256)
b = random.randrange(256)

print(f"a: {a} (int), {hexadecimal(a, pre='')} (hex), {binary(a, pre='')} (bin)")
print(f"b: {b} (int), {hexadecimal(b, pre='')} (hex), {binary(b, pre='')} (bin)")

xored = a ^ b
print(f"x: {xored} (int), {hexadecimal(xored, pre='')} (hex), {binary(xored, pre='')} (bin)")

print(f"\n{binary(a, pre='')}\n+\n{binary(b, pre='')}\n=\n{binary(xored, pre='')}")

a: 148 (int), 94 (hex), 10010100 (bin)
b: 174 (int), ae (hex), 10101110 (bin)
x: 58 (int), 3a (hex), 00111010 (bin)

10010100
+
10101110
=
00111010


### Bytes random generator <a class="anchor" id="brg"></a>

We need a pseudorandom genertor for bytes, let's code it!

In Python we have the ```state``` variable to store the state of the pseudorandom generator in ```random``` package.

In [3]:
# in random python pacakge we have the "state" of the PRG
random.seed(10)
state = random.getstate()

print([random.randrange(256) for _ in range(10)])

random.setstate(state)
print([random.randrange(256) for _ in range(10)])


[16, 219, 247, 7, 105, 236, 251, 142, 82, 17]
[16, 219, 247, 7, 105, 236, 251, 142, 82, 17]


In [4]:
def PseudoRandomBytes(state: tuple, l: int) -> (tuple, bytes):
    """
    Generates a stream of pseudorandom bytes
    Input:
        - state: a state for the python random pacakge (random.getstate())
        - l: length of the pseudorandom stream of bytes
    Returns:
        - state: the current state of the random
        - bytestream: a bytes class of lenght l
    """
    random.setstate(state)
    prng = []
    
    while len(prng) < l:
        prng.append(hexadecimal(random.randrange(256)))
    
    return random.getstate(), bytes([int(x, 0) for x in prng])

In [5]:
state = random.getstate()
l = 20

new_state, prng = PseudoRandomBytes(state, l)
print(prng)

b'\xfa\xa7&\x7f\xb8\x16\xd7G\xb5\xc3\xd7\x91\x86\xe9Y\x9b\xb9D\xe9z'


## Practical demonstration with python objects <a class="anchor" id="practical"></a>

Let's go practical in this section!. We will build a python object called `Party` that can encrypt and decrypt information keeping track of the state of the pseudorandom generator.

In [6]:
class Party:
    def __init__(self, state: tuple):
        self._state = state
        
    def encrypt_decrypt(self, m: bytes) -> bytes:
        new_state, random_bytes = PseudoRandomBytes(self._state, len(m))
        self._state = new_state
        
        return bytes([a ^ b for a, b in zip(m, random_bytes)])

In [7]:
state = random.getstate()

alice = Party(state)
bob = Party(state)

In [8]:
m = b"Hi Bob. How are you doing?"
ctx = alice.encrypt_decrypt(m)
m2 = bob.encrypt_decrypt(ctx)

print(f"message:\n\t{m}\n")
print(f"ciphertext:\n\t{ctx}\n")
print(f"recovered_message:\n\t{m2}\n")

message:
	b'Hi Bob. How are you doing?'

ciphertext:
	b'\xa9\xa96@\x17&M\xbb\xf3\x15\xd7\xc6\xbe\x82D\x86)\x1d\xa6Zv\x7f\x97\xf4C\x16'

recovered_message:
	b'Hi Bob. How are you doing?'



In [9]:
m = b"I'm good, thank you!. How do you do?"
ctx = bob.encrypt_decrypt(m)
m2 = alice.encrypt_decrypt(ctx)


print(f"message:\n\t{m}\n")
print(f"ciphertext:\n\t{ctx}\n")
print(f"recovered_message:\n\t{m2}\n")

message:
	b"I'm good, thank you!. How do you do?"

ciphertext:
	b'\x05\xe3\xd2l^^\x8d1M\x92\xa9\xbc\x85\x13\xe7i"R\xfd\xc8\xb4t\x115\x82\x91\xc3\xb1R{y\xdc\x83\x18G\xb9'

recovered_message:
	b"I'm good, thank you!. How do you do?"

